<b>Name:</b> cum_gpa_by_term.ipynb <br>
<b>Author:</b> Yifeng Song <br>
<b>Last Modified</b>: 08/23/2020 <br>
<b>Purpose:</b> The raw VCCS data report the student GPA by student x college x term. This script will calculate the aggregated cumulative GPA value by student x term, which will be used in the subsequent steps of constructing the predictors of the training/validation sample for the predictive models.

In [1]:
import os
import getpass
import pandas as pd
import numpy as np
import gc
import subprocess

home_dir = os.path.expanduser('~')
fpath = os.path.join(home_dir, 'C:\\Users\\{}\\Box Sync\\VCCS data partnership\\vccs_project_data\\intermediate_files\\ys8mz\\{}'.format(getpass.getuser()))

In [2]:
# Load the merged VCCS GPA data and keep the useful columns
df = pd.read_stata(os.path.join(fpath, "Merged_GPA.dta"))
df = df.loc[:,['vccsid', 'institution', 'collnum', 'strm', 'cum_gpa', 'tot_taken_prgrss']]
all_terms = sorted(np.unique(df.strm))
print(all_terms)

[2003, 2004, 2012, 2013, 2014, 2022, 2023, 2024, 2032, 2033, 2034, 2042, 2043, 2044, 2052, 2053, 2054, 2062, 2063, 2064, 2072, 2073, 2074, 2082, 2083, 2084, 2092, 2093, 2094, 2102, 2103, 2104, 2112, 2113, 2114, 2122, 2123, 2124, 2132, 2133, 2134, 2142, 2143, 2144, 2152, 2153, 2154, 2162, 2163, 2164, 2172, 2173, 2174, 2182, 2183, 2184, 2192, 2193, 2194, 2202, 2203, 2204]


In [3]:
# Construct the nested dictionary 'cum_gpa_dict', with the primary key being the student x collegename,
# and the secondary key being the term number. And the values of inner dictionaries correspond to 
# the cumulative credits earned and the cumulative GPA at each college the student attended
cum_gpa_dict = {}
for i in range(df.shape[0]):
    if i % 1e5 == 0: # keep running garbage collection in every 100000 observations processed to prevent memory leak
        print(i)
        gc.collect()
    stuid = df.vccsid.iloc[i]
    college = df.institution.iloc[i]
    stuid_college = stuid + "-" + college
    term = df.strm.iloc[i]
    credits_gpa_pair = (df.tot_taken_prgrss.iloc[i], df.cum_gpa.iloc[i])
    if stuid_college not in cum_gpa_dict:
        cum_gpa_dict[stuid_college] = {term:credits_gpa_pair}
    else:
        cum_gpa_dict[stuid_college][term] = credits_gpa_pair
del df
gc.collect()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

14

In [4]:
# If a student didn't attend the college during the first semester (according to the merged data),
# the values corresponding to the terms prior to the student's actual first term should be (0,0),
# meaning 0 credits attempted and 0 GPA; a new nested dictionary "cum_gpa_dict_2" will be created;
# If a student stopped out during a certain semester, the cumulative credits and GPA will be the
# values of the most recent prior semester -- this will make it easier to calculate aggregated cumulative GPA
first_term = all_terms[0]
cum_gpa_dict_2 = {}
i = 0
for k,v in cum_gpa_dict.items():
    if i % 1e5 == 0:
        print(i)
        gc.collect()
    new_v = v.copy()
    if first_term not in v:
        new_v[first_term] = (0,0)
    for indx,t in enumerate(all_terms[:-1]):
        crnt_term = all_terms[indx+1]
        if crnt_term not in new_v:
            new_v[crnt_term] = new_v[t]
    cum_gpa_dict_2[k] = new_v.copy()
    i += 1
del cum_gpa_dict
gc.collect()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000


0

In [5]:
# Create a new nested dictionary "cum_gpa_dict_3", which has student_id as the primary key,
# and term number as the secondary, and the values in the inner dictionaries correspond to
# the list of all pairs of cumulative credits earned & cumulative GPA of the student during
# the term, with each pair corresponding to each college the student attended during that term
cum_gpa_dict_3 = {}
i = 0
for k,v in cum_gpa_dict_2.items():
    if i % 1e5 == 0:
        print(i)
        gc.collect()
    stuid = k.split("-")[0]
    if stuid not in cum_gpa_dict_3:
        cum_gpa_dict_3[stuid] = {k2:[v2] for k2,v2 in v.items()}
    else:
        for k2,v2 in v.items():
            cum_gpa_dict_3[stuid][k2].append(v2)
    i += 1
del cum_gpa_dict_2
gc.collect()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000


0

In [6]:
def calc_agg_cum_gpa(l):
    # This function calculates the aggregated cumulative GPA of the student during each term:
    # If the student attended more than one institution, the aggregated cumulative GPA will be
    # the weighted average of the cumulative GPA of all colleges student has ever attended so far,
    # even if the student is only actively enrolled in one college during a certain term
    a = np.array(l)
    if a.shape[0] == 1:
        if a[0,0] > 0:
            r = a[0,1]
        else:
            r = np.nan
    else:
        a = np.array(l)
        a[:,1] = a[:,0]*a[:,1]
        b = np.sum(a,axis=0)
        if b[0] == 0:
            r = np.nan
        else:
            r = b[1]/b[0]
    if pd.isnull(r):
        return r
    else:
        return round(r,3)

In [7]:
# create the nested dictionary "agg_cum_gpa", which has student_id as the primary key and term number as the secondary key,
# and there is one aggregated GPA value corresponding to each student x term
agg_cum_gpa = {}
i = 0
for k,v in cum_gpa_dict_3.items():
    if i % 1e5 == 0:
        print(i)
        gc.collect()
    agg_cum_gpa[k] = {k2:calc_agg_cum_gpa(v2) for k2,v2 in v.items()}
    i += 1
del cum_gpa_dict_3
gc.collect()

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000


0

In [8]:
# Final cleanup and transform the aggregated cumulative GPA data from dictionary format into tabular format,
# save the tabular data to file in the .dta file format
final_df = pd.DataFrame.from_dict(agg_cum_gpa, orient="index")
final_df = final_df.loc[:,sorted(final_df.columns.values)] # The column names are order by term
gc.collect()
final_df.columns = ["term_" + str(cn) for cn in final_df.columns.values] # Add prefix "term_" to column names
final_df.reset_index(inplace=True)
final_df.rename(columns = {'index':'vccsid'}, inplace=True)
final_df.to_stata(os.path.join(fpath, "agg_cum_gpa_by_term.dta"), write_index=False)